In [ ]:
import pandas as pd
import os
import shapely.geometry as sg
data_directory = os.path.join('..', 'data', 'ptv', '20240224')

In [ ]:
branch_ids = ['1', '2', '3', '4', '5', '6', '7', '8', '10', '11']
table_names = ['stop_times', 'stops', 'trips', 'routes', 'calendar', 'calendar_dates', 'agency', 'shapes']

In [ ]:
def get_df(branch_id, table_name):
    files = [os.path.join(data_directory, f) for f in os.listdir(data_directory) if f.split('-')[1] == str(branch_id) and f.split('-')[2] == table_name]
    if len(files) == 0:
        return None
    return pd.concat([pd.read_csv(f) for f in files])

In [ ]:
df = {branch_id: {table_name: get_df(branch_id, table_name) for table_name in table_names} for branch_id in branch_ids}

In [ ]:
bid = '3'

In [ ]:
df_trams = df['3']['stop_times'].groupby('trip_id')['stop_id'].apply(lambda x: x.to_numpy()).to_frame(name='stops_sequence').reset_index()
df_trams = pd.merge(df['3']['trips'], df_trams, on='trip_id')
df_trams = pd.merge(df['3']['routes'], df_trams, on='route_id')
df_trams['stops_count'] = df_trams['stops_sequence'].apply(lambda x: len(x))
df_trams = df_trams.sort_values(by=['route_short_name', 'stops_count'], ascending=False).drop_duplicates(subset=['route_short_name', 'trip_headsign', 'direction_id'], keep='first')

df_tram_stops = pd.merge(df['3']['stop_times'], df['3']['trips'], on='trip_id')
df_tram_stops = pd.merge(df_tram_stops, df['3']['routes'], on='route_id')
df_tram_stops  = df_tram_stops.groupby(['route_short_name', 'trip_headsign', 'direction_id'])['stop_id'].apply(lambda x: x.unique()).to_frame(name='stops').reset_index()

df_trams = df_trams.merge(df_tram_stops, on=['route_short_name', 'trip_headsign', 'direction_id'])

df_trams['stops_count_2'] = df_trams['stops'].apply(lambda x: len(x))
df_trams['stops_set_1'] = df_trams['stops_sequence'].apply(lambda x: sorted(x))
df_trams['stops_set_2'] = df_trams['stops'].apply(lambda x: sorted(x))

df_trams['stops_set_1_size'] = df_trams['stops_set_1'].apply(lambda x: len(set(x)))
df_trams['stops_set_2_size'] = df_trams['stops_set_2'].apply(lambda x: len(set(x)))

In [ ]:
df_trams[df_trams['stops_set_1'] != df_trams['stops_set_2']].iloc[2]['stops_sequence']

In [ ]:
df_routes_full = pd.merge(df['3']['trips'], df['3']['routes'], on='route_id', how='left')

In [ ]:
df_routes_full = pd.merge(df_routes_full, df['3']['stop_times'], on='trip_id', how='left')

In [ ]:
df_trams['anomalies'] = df_trams.apply(lambda x: set(x['stops']) - set(x['stops_sequence']), axis=1)

In [ ]:
df_anomalies = df_trams[df_trams['anomalies'].apply(lambda x: len(x) > 0)][['route_short_name', 'trip_headsign', 'direction_id', 'anomalies']]

In [ ]:
df_at = df_anomalies.explode('anomalies').merge(df_routes_full, left_on=['route_short_name', 'trip_headsign', 'direction_id', 'anomalies'], right_on=['route_short_name', 'trip_headsign', 'direction_id', 'stop_id']).sort_values(by=['trip_id', 'stop_sequence'], ascending=True)
df_at = df_at.groupby('trip_id')['stop_id'].apply(lambda x: x.to_numpy()).to_frame(name='stops_sequence').reset_index()

In [ ]:
df_at = df_at.groupby('trip_id')['stop_id'].apply(lambda x: x.to_numpy()).to_frame(name='stops_sequence').reset_index()

In [ ]:
df_at = pd.merge(df['3']['trips'], df_at, on='trip_id')

In [ ]:
df_at['stops_count'] = df_at['stops_sequence'].apply(lambda x: len(x))

In [ ]:
df_at = pd.merge(df['3']['routes'], df_at, on='route_id')

In [ ]:
df_at = df_at.sort_values(by=['route_short_name', 'stops_count'], ascending=False).drop_duplicates(subset=['route_short_name', 'trip_headsign', 'direction_id'], keep='first').merge(df_anomalies, on=['route_short_name', 'trip_headsign', 'direction_id'], how='left')

In [ ]:
df_at[df_at.apply(lambda x: set(x['anomalies']) - set(x['stops_sequence']), axis=1).apply(lambda x: len(x) > 0)]

In [ ]:
df_at = df_at[['route_short_name', 'trip_headsign', 'direction_id', 'trip_id']]

In [ ]:
df_at = df_at.merge(df['3']['stop_times'], on='trip_id')

In [ ]:
df_at = df_at.groupby('trip_id')['stop_id'].apply(lambda x: x.to_numpy()).to_frame(name='stops_sequence').reset_index()

In [ ]:
df_at = pd.merge(df['3']['trips'], df_at, on='trip_id')
df_at = pd.merge(df['3']['routes'], df_at, on='route_id')

In [ ]:
df_at = pd.concat([df_at, df_trams])

In [ ]:
df_at.drop(columns=['anomalies', 'stops_count', 'stops', 'stops_count_2', 'stops_set_1', 'stops_set_2', 'stops_set_1_size', 'stops_set_2_size'], inplace=True)

In [ ]:
df_at['stops_count'] = df_at['stops_sequence'].apply(lambda x: len(x))

In [ ]:
df_at.sort_values(by=['route_short_name', 'trip_headsign', 'direction_id', 'stops_count'], ascending=False, inplace=True)
df_at.to_csv('trips.csv', index=False)

In [ ]:
df['3']['shapes']['point'] = df['3']['shapes'].apply(lambda x: sg.Point(x['shape_pt_lon'], x['shape_pt_lat']), axis=1)

In [ ]:
df_lines = df['3']['shapes'].groupby('shape_id')['point'].apply(lambda x: sg.LineString(x.to_numpy())).to_frame(name='line').reset_index()

In [ ]:
df['3']['stops']['stop_pt'] = df['3']['stops'].apply(lambda x: sg.Point(x['stop_lon'], x['stop_lat']), axis=1)

In [ ]:
df_geo = pd.merge(df['3']['stop_times'], df['3']['trips'][['trip_id', 'shape_id']], on='trip_id')
df_geo = pd.merge(df_geo, df_lines, on='shape_id')
df_geo = df_geo[['shape_id', 'stop_id']].drop_duplicates()
df_geo = pd.merge(df_geo, df_lines, on='shape_id')
df_geo = pd.merge(df_geo, df['3']['stops'][['stop_id', 'stop_pt']], on='stop_id')

In [ ]:
df_geo[df_geo.apply(lambda x: x['line'].contains(x['stop_pt']), axis=1)]

In [ ]:
bid = '4'
dfrt = pd.merge(df[bid]['routes'], df[bid]['trips'], on='route_id')
# assert dfrt.apply(lambda x: str(x['route_short_name']) in x['route_id'], axis=1).all()


In [ ]:
assert dfrt.apply(lambda x: str(x['route_id']) in str(x['shape_id']), axis=1).all()
assert dfrt.apply(lambda x: x['shape_id'] in x['trip_id'], axis=1).all()


In [ ]:
bid = '5'
dfrt = pd.merge(df[bid]['routes'], df[bid]['trips'], on='route_id')
dfrt.dropna(subset=['shape_id'], inplace=True)
dfrt['shape_id_x'] = dfrt['shape_id'].apply(lambda x: x.split('.'))
shape_id_x = dfrt['shape_id_x'].apply(len).unique()
assert len(shape_id_x) == 1
shape_id_count = shape_id_x[0]
for i in range(shape_id_count):
    dfrt[f'shape_id_{i}'] = dfrt['shape_id_x'].apply(lambda x: x[i])
dfrt[['shape_id_1', 'shape_id_2']].sort_values(by=['shape_id_1', 'shape_id_2'], ascending=True).value_counts()


In [ ]:
bid = '11'
dfrt = pd.merge(df[bid]['routes'], df[bid]['trips'], on='route_id')
dfrt.groupby('route_id')['direction_id'].unique().apply(len).value_counts()

In [ ]:
dfrt[['shape_id_1', 'shape_id_2']].value_counts()

In [ ]:
bid = '11'
df[bid]['routes'][df[bid]['routes'].apply(lambda x: str(x['route_short_name']) not in x['route_id'], axis=1)]

In [ ]:
dfrt[['direction_id', 'trip_id_4']].value_counts()

In [ ]:
df['3']['trips']['direction_id'].unique()

In [ ]:
df['5']['trips']['direction_id']

In [ ]:
assert dfrt.apply(lambda x: x['shape_id'] in x['trip_id'], axis=1).all()

In [ ]:
dfrt['shape_id_1'].unique()

In [ ]:
assert dfrt['shape_id_x'].apply(len).nunique()

In [ ]:
dfrt['trip_id_x'].apply(len).unique()

In [ ]:
dfrt['shape_id_2'] = dfrt['shape_id_1'].apply(lambda x: x[1])
dfrt['trip_id_2'] = dfrt['trip_id_1'].apply(lambda x: x[1])


In [ ]:
pd.merge(df['3']['routes'], df['3']['trips'], on='route_id').groupby(['route_short_name', 'trip_headsign', 'direction_id'])['shape_id'].unique()